# Lab: Use Spark Dataframes for ETL

In this lab, you will use Spark dataframes to process some data

--> 1. This lab uses the zip archive webpage.zip from Blackboard. You should either use the reusable notebook you wrote
in a previous lab to unzip this and place the resulting webpage directory in DBFS's/FileStore/ or you should follow
the steps from a previous lab to do this manually.

--> 2. View the beginning of the part-00000 file. Don't forget that the splitlines() method will split your string into a
list by splitting at line breaks. You can therefore make your output much more presentable by passing through this
list using something similar to:

In [0]:
dbutils.fs.head("dbfs:/FileStore/tables/extracts/webpage/part-m-00000")
#print(dbutils.fs.head("dbfs:/FileStore/tables/extracts/webpage/part-m-00000"))

Out[2]: '1\tsorrento_f00l_sales.html\ttheme.css,code.js,sorrento_f00l.jpg\n2\ttitanic_2100_sales.html\ttheme.css,code.js,titanic_2100.jpg\n3\tmeetoo_3.0_sales.html\ttheme.css,code.js,meetoo_3.0.jpg\n4\tmeetoo_3.1_sales.html\ttheme.css,code.js,meetoo_3.1.jpg\n5\tifruit_1_sales.html\ttheme.css,code.js,ifruit_1.jpg\n6\tifruit_3_sales.html\ttheme.css,code.js,ifruit_3.jpg\n7\tifruit_2_sales.html\ttheme.css,code.js,ifruit_2.jpg\n8\tifruit_5_sales.html\ttheme.css,code.js,ifruit_5.jpg\n9\ttitanic_1000_sales.html\ttheme.css,code.js,titanic_1000.jpg\n10\tmeetoo_1.0_sales.html\ttheme.css,code.js,meetoo_1.0.jpg\n11\tsorrento_f21l_sales.html\ttheme.css,code.js,sorrento_f21l.jpg\n12\tifruit_4_sales.html\ttheme.css,code.js,ifruit_4.jpg\n13\tsorrento_f23l_sales.html\ttheme.css,code.js,sorrento_f23l.jpg\n'

Using dbutils.fs.head(<filename>).splitlines() to do the same thing as above

In [0]:
for line in dbutils.fs.head("dbfs:/FileStore/tables/extracts/webpage/part-m-00000").splitlines():
  print(line)

1	sorrento_f00l_sales.html	theme.css,code.js,sorrento_f00l.jpg
2	titanic_2100_sales.html	theme.css,code.js,titanic_2100.jpg
3	meetoo_3.0_sales.html	theme.css,code.js,meetoo_3.0.jpg
4	meetoo_3.1_sales.html	theme.css,code.js,meetoo_3.1.jpg
5	ifruit_1_sales.html	theme.css,code.js,ifruit_1.jpg
6	ifruit_3_sales.html	theme.css,code.js,ifruit_3.jpg
7	ifruit_2_sales.html	theme.css,code.js,ifruit_2.jpg
8	ifruit_5_sales.html	theme.css,code.js,ifruit_5.jpg
9	titanic_1000_sales.html	theme.css,code.js,titanic_1000.jpg
10	meetoo_1.0_sales.html	theme.css,code.js,meetoo_1.0.jpg
11	sorrento_f21l_sales.html	theme.css,code.js,sorrento_f21l.jpg
12	ifruit_4_sales.html	theme.css,code.js,ifruit_4.jpg
13	sorrento_f23l_sales.html	theme.css,code.js,sorrento_f23l.jpg

## Observations - 

The columns are id, an integer, followed by two string columns: webpage and associated files. Note that the data in
the last column, the associated les, is a comma-delimited string. We're going to extract the data in that column, split the
string and create a new dataset containing each webpage and its associated les in separate rows.

## Create a dataframe

You will now explore ways to make dataframes: both from input text and from other dataframes.

--> 3. Create a new dataframe called webpages from the webpage data. You should try creating this in three different ways:

##(a) Using spark.createDataFrame()
where you create a schema and an RDD and use these to create the dataframe.

In [0]:
from pyspark.sql.types import *

mySchema = StructType( [
                        StructField("id", IntegerType()),
                        StructField("webpage", StringType()),
                        StructField("associated_files", StringType())
                          ] )

webRDD = sc.textFile("dbfs:/FileStore/tables/extracts/webpage/*"). \
         map( lambda line : line.split()).  \
         map( lambda values :[int(values[0]),values[1],values[2]])

webDF = spark.createDataFrame(webRDD, mySchema)
webDF.show(5)

# Point to note - .split() function directly checks for linebreaks
# webRDD.take(5)


+---+--------------------+--------------------+
 id| webpage| associated_files|
+---+--------------------+--------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|
 2|titanic_2100_sale...|theme.css,code.js...|
 3|meetoo_3.0_sales....|theme.css,code.js...|
 4|meetoo_3.1_sales....|theme.css,code.js...|
 5| ifruit_1_sales.html|theme.css,code.js...|
+---+--------------------+--------------------+
only showing top 5 rows

## (b) Using spark.read.csv()
Don't forget that you will need to change the default delimiter.

In [0]:
webDF_read = spark.read.csv("dbfs:/FileStore/tables/extracts/webpage/part-m-*", sep="\t")
webDF_read.printSchema()

root
-- _c0: string (nullable = true)
-- _c1: string (nullable = true)
-- _c2: string (nullable = true)

In [0]:
webDF_read.show(5)

+---+--------------------+--------------------+
_c0| _c1| _c2|
+---+--------------------+--------------------+
 14|titanic_2200_sale...|theme.css,code.js...|
 15|ronin_novelty_not...|theme.css,code.js...|
 16|titanic_2500_sale...|theme.css,code.js...|
 17|ronin_novelty_not...|theme.css,code.js...|
 18|ronin_novelty_not...|theme.css,code.js...|
+---+--------------------+--------------------+
only showing top 5 rows

##(c) Using .toDF() method from an RDD.

In [0]:
webDF_Method = webRDD.toDF(schema=mySchema)
webDF_Method.show(5)

+---+--------------------+--------------------+
 id| webpage| associated_files|
+---+--------------------+--------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|
 2|titanic_2100_sale...|theme.css,code.js...|
 3|meetoo_3.0_sales....|theme.css,code.js...|
 4|meetoo_3.1_sales....|theme.css,code.js...|
 5| ifruit_1_sales.html|theme.css,code.js...|
+---+--------------------+--------------------+
only showing top 5 rows

In [0]:
webDF_Method_NoSch = webRDD.toDF()
webDF_Method_NoSch.show(5)

+---+--------------------+--------------------+
 _1| _2| _3|
+---+--------------------+--------------------+
 1|sorrento_f00l_sal...|theme.css,code.js...|
 2|titanic_2100_sale...|theme.css,code.js...|
 3|meetoo_3.0_sales....|theme.css,code.js...|
 4|meetoo_3.1_sales....|theme.css,code.js...|
 5| ifruit_1_sales.html|theme.css,code.js...|
+---+--------------------+--------------------+
only showing top 5 rows

--> 4. Examine the schema of the new DataFrame by calling webpages.printSchema(). If your columns don't have the
expected names, ensure that you rename them (e.g. using withColumnRenamed). Also, you should check that the rst
column has an integer type. If it does not, you need to change this: one way to do it is using withColumn and cast.

In [0]:
webDF_Method.printSchema()

root
-- id: integer (nullable = true)
-- webpage: string (nullable = true)
-- associated_files: string (nullable = true)

In [0]:
webDF_Method.withColumnRenamed("id", "ID")

Out[10]: DataFrame[ID: int, webpage: string, associated_files: string]

In [0]:
webDF_Method.printSchema()

root
-- id: integer (nullable = true)
-- webpage: string (nullable = true)
-- associated_files: string (nullable = true)

In [0]:
webDF_Method_NoSch.printSchema()

root
-- _1: long (nullable = true)
-- _2: string (nullable = true)
-- _3: string (nullable = true)

In [0]:
webDF_Method_NoSch["_1"].cast("integer")

Out[13]: Column<b'CAST(_1 AS INT)'>

In [0]:
webDF_Method_NoSch.withColumn("_1", webDF_Method_NoSch["_1"].cast("Integer")).withColumnRenamed("_1", "ID").withColumnRenamed("_2", "Webpage")

Out[14]: DataFrame[ID: int, Webpage: string, _3: string]

In [0]:
webDF_Method_NoSch.printSchema()

root
-- _1: long (nullable = true)
-- _2: string (nullable = true)
-- _3: string (nullable = true)

RDDs are immutable, so obviously DFs also become immutable

--> In order to change the DF, just use an equal to operator and assign

In [0]:
webDF_Method_NoSch = webDF_Method_NoSch.withColumn("_1", webDF_Method_NoSch["_1"].cast("Integer")).withColumnRenamed("_1", "ID").withColumnRenamed("_2", "Webpage")

--> 5. Check that your revised webpage dataframes all have the same schema.

In [0]:
webDF_Method_NoSch.printSchema()

root
-- ID: integer (nullable = true)
-- Webpage: string (nullable = true)
-- _3: string (nullable = true)

--> 6. Create a new DataFrame by selecting the webpage and associated files columns from the existing DataFrame.

In [0]:
webDF_sliced = webDF_Method.select("webpage", "associated_files")
webDF_sliced.show(5)

+--------------------+--------------------+
 webpage| associated_files|
+--------------------+--------------------+
sorrento_f00l_sal...|theme.css,code.js...|
titanic_2100_sale...|theme.css,code.js...|
meetoo_3.0_sales....|theme.css,code.js...|
meetoo_3.1_sales....|theme.css,code.js...|
 ifruit_1_sales.html|theme.css,code.js...|
+--------------------+--------------------+
only showing top 5 rows

--> 7. In order to manipulate the data using Spark, convert the DataFrame into a Pair RDD using the map method. The
input into the map method is a Row object. They key is the webpage value, and the value is the associated files
string.

In [0]:
webRDD_sliced = webDF_sliced.rdd.map(lambda line: (line.webpage, line.associated_files))
webRDD_sliced.take(5)

Out[19]: [('sorrento_f00l_sales.html', 'theme.css,code.js,sorrento_f00l.jpg'),
 ('titanic_2100_sales.html', 'theme.css,code.js,titanic_2100.jpg'),
 ('meetoo_3.0_sales.html', 'theme.css,code.js,meetoo_3.0.jpg'),
 ('meetoo_3.1_sales.html', 'theme.css,code.js,meetoo_3.1.jpg'),
 ('ifruit_1_sales.html', 'theme.css,code.js,ifruit_1.jpg')]

--> 8. Now that you have an RDD, you can use the familiar flatMapValues transformation to split and extract the lenames
in the associated files column.

In [0]:
Files_RDD = webRDD_sliced.flatMapValues(lambda line: line.split(","))
Files_RDD.take(15)

Out[30]: [('sorrento_f00l_sales.html', 'theme.css'),
 ('sorrento_f00l_sales.html', 'code.js'),
 ('sorrento_f00l_sales.html', 'sorrento_f00l.jpg'),
 ('titanic_2100_sales.html', 'theme.css'),
 ('titanic_2100_sales.html', 'code.js'),
 ('titanic_2100_sales.html', 'titanic_2100.jpg'),
 ('meetoo_3.0_sales.html', 'theme.css'),
 ('meetoo_3.0_sales.html', 'code.js'),
 ('meetoo_3.0_sales.html', 'meetoo_3.0.jpg'),
 ('meetoo_3.1_sales.html', 'theme.css'),
 ('meetoo_3.1_sales.html', 'code.js'),
 ('meetoo_3.1_sales.html', 'meetoo_3.1.jpg'),
 ('ifruit_1_sales.html', 'theme.css'),
 ('ifruit_1_sales.html', 'code.js'),
 ('ifruit_1_sales.html', 'ifruit_1.jpg')]

--> 9. Create a new DataFrame from the RDD.

In [0]:
Files_DF = Files_RDD.toDF()
Files_DF.show(15)

+--------------------+-----------------+
 _1| _2|
+--------------------+-----------------+
sorrento_f00l_sal...| theme.css|
sorrento_f00l_sal...| code.js|
sorrento_f00l_sal...|sorrento_f00l.jpg|
titanic_2100_sale...| theme.css|
titanic_2100_sale...| code.js|
titanic_2100_sale...| titanic_2100.jpg|
meetoo_3.0_sales....| theme.css|
meetoo_3.0_sales....| code.js|
meetoo_3.0_sales....| meetoo_3.0.jpg|
meetoo_3.1_sales....| theme.css|
meetoo_3.1_sales....| code.js|
meetoo_3.1_sales....| meetoo_3.1.jpg|
 ifruit_1_sales.html| theme.css|
 ifruit_1_sales.html| code.js|
 ifruit_1_sales.html| ifruit_1.jpg|
+--------------------+-----------------+
only showing top 15 rows

--> 10. Call printSchema on the new DataFrame.

In [0]:
Files_DF.printSchema()

root
-- _1: string (nullable = true)
-- _2: string (nullable = true)

--> 11. If you ended up with generic names for columns, such as 1 and 2, create a new DataFrame by renaming the columns
to reflect the data they hold using the withColumnRenamed method to rename the two columns.

In [0]:
Files_DF = Files_DF.withColumnRenamed("_1", "webpage").withColumnRenamed("_2", "associated_files")


--> 12. Call printSchema to conrm that the new DataFrame has the correct column names.

In [0]:
Files_DF.printSchema()

root
-- webpage: string (nullable = true)
-- associated_files: string (nullable = true)

--> 13. Your final DataFrame contains the processed data, so save it in Parquet format (the default) in /FileStore/webpage files

In [0]:
# df.select("name", "favorite_color").write.save("namesAndFavColors.parquet")

Files_DF.write.save("/FileStore/webpage_files/")

--> 14. Check the les have been created.

In [0]:
dbutils.fs.ls("/FileStore/webpage_files")

Out[49]: [FileInfo(path='dbfs:/FileStore/webpage_files/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/FileStore/webpage_files/_committed_5635270027111026027', name='_committed_5635270027111026027', size=420),
 FileInfo(path='dbfs:/FileStore/webpage_files/_started_5635270027111026027', name='_started_5635270027111026027', size=0),
 FileInfo(path='dbfs:/FileStore/webpage_files/part-00000-tid-5635270027111026027-ad3dbd85-ffc6-4cf3-93b5-f021c4d37e22-27-1-c000.snappy.parquet', name='part-00000-tid-5635270027111026027-ad3dbd85-ffc6-4cf3-93b5-f021c4d37e22-27-1-c000.snappy.parquet', size=1163),
 FileInfo(path='dbfs:/FileStore/webpage_files/part-00001-tid-5635270027111026027-ad3dbd85-ffc6-4cf3-93b5-f021c4d37e22-28-1-c000.snappy.parquet', name='part-00001-tid-5635270027111026027-ad3dbd85-ffc6-4cf3-93b5-f021c4d37e22-28-1-c000.snappy.parquet', size=1179),
 FileInfo(path='dbfs:/FileStore/webpage_files/part-00002-tid-5635270027111026027-ad3dbd85-ffc6-4cf3-93b5-f021c4d37e22-29-1-c000.snappy.parquet', name='part-00002-tid-5635270027111026027-ad3dbd85-ffc6-4cf3-93b5-f021c4d37e22-29-1-c000.snappy.parquet', size=1217),
 FileInfo(path='dbfs:/FileStore/webpage_files/part-00003-tid-5635270027111026027-ad3dbd85-ffc6-4cf3-93b5-f021c4d37e22-30-1-c000.snappy.parquet', name='part-00003-tid-5635270027111026027-ad3dbd85-ffc6-4cf3-93b5-f021c4d37e22-30-1-c000.snappy.parquet', size=1188)]